# python_subdict

The intent of this library is to make it easy to extract subdicts from python dicts by just specifying which keys are needed, in a dotted-syntax.

The ipython notebook version of this documentation is [here](https://github.com/victor-o-silva/python_subdict/blob/master/README.ipynb). Please check it there.

As an example, let's say that we have the following dict:

In [1]:
d = {
    'a': 'A',
    'b': 'B',
    'c': 'C',
    'd': {
        'x': 'D_X',
        'y': 'D_Y',
        'z': {
            'I': 'D_Z_I',
            'II': {
                '1': 'D_Z_II_1',
                '2': 'D_Z_II_2'
            },
            'III': 'D_Z_III'
        }
    }
}

If we need only the keys `'a'` and `'d'`, we can do this:

In [2]:
from extract_subdict import extract_subdict  # The main function of the library

In [3]:
from pprint import pprint  # Just for a nice presentation here

In [4]:
pprint( extract_subdict(d, ['a', 'd']) )

{'a': 'A',
 'd': {'x': 'D_X',
       'y': 'D_Y',
       'z': {'I': 'D_Z_I',
             'II': {'1': 'D_Z_II_1', '2': 'D_Z_II_2'},
             'III': 'D_Z_III'}}}


We can also specify 'subkeys' by using a dotted-syntax:

In [5]:
pprint( extract_subdict(d, ['a', 'd.x', 'd.z']) )

{'a': 'A',
 'd': {'x': 'D_X',
       'z': {'I': 'D_Z_I',
             'II': {'1': 'D_Z_II_1', '2': 'D_Z_II_2'},
             'III': 'D_Z_III'}}}


The dotted-syntax can have any needed level of depth:

In [6]:
pprint( extract_subdict(d, ['a', 'd.z.II.1']) )

{'a': 'A', 'd': {'z': {'II': {'1': 'D_Z_II_1'}}}}


# Specifying invalid keys behavior

Let's consider the following dict from now:

In [7]:
person = {
    'name': 'John Frusciante',
    'birth': '1970-03-05',
    'city': {
        'name': 'New York City',
        'state': {'name': 'New York', 'country': 'USA'}
    },
    'albums': [
        {
            'year': 2001,
            'name': 'To Record Only Water For Ten Days',
            'label': {
                'name': 'Warner Bros Records',
                'link': 'https://en.wikipedia.org/wiki/Warner_Bros._Records'
            }
        },
        {
            'year': 2004,
            'name': 'Shadows Collide With People',
            'label': {
                'name': 'Warner Bros Records',
                'link': 'https://en.wikipedia.org/wiki/Warner_Bros._Records'
            }
        },
        {
            'year': 2009,
            'name': 'The Empyrean',
            'label': {
                'name': 'Record Collection',
                'link': 'https://en.wikipedia.org/wiki/Record_Collection'
            }
        }
    ]
}

By default, invalid keys passed to the `extract_subdict` function are ignored:

In [8]:
extract_subdict(person, ['name', 'birth', 'hair_color'])  # 'hair_color' is not present in the `person` dict

{'birth': '1970-03-05', 'name': 'John Frusciante'}

However, by passing `True` to the `strict` parameter of the function, invalid keys will raise a `KeyError` exception:

In [9]:
extract_subdict(person, ['name', 'birth', 'hair_color'], strict=True)

KeyError: 'hair_color'

# Successive extractions

Extracting only `'name'` and `'albums'` from the `person` dict:

In [10]:
subdict = extract_subdict(person, ['name', 'albums'])

In [11]:
pprint(subdict)

{'albums': [{'label': {'link': 'https://en.wikipedia.org/wiki/Warner_Bros._Records',
                       'name': 'Warner Bros Records'},
             'name': 'To Record Only Water For Ten Days',
             'year': 2001},
            {'label': {'link': 'https://en.wikipedia.org/wiki/Warner_Bros._Records',
                       'name': 'Warner Bros Records'},
             'name': 'Shadows Collide With People',
             'year': 2004},
            {'label': {'link': 'https://en.wikipedia.org/wiki/Record_Collection',
                       'name': 'Record Collection'},
             'name': 'The Empyrean',
             'year': 2009}],
 'name': 'John Frusciante'}


Extracting only the `'name'` of each album:

In [18]:
for index in range(len(subdict['albums'])):
    subdict['albums'][index] = extract_subdict(subdict['albums'][index], ['name'])

In [19]:
pprint(subdict)

{'albums': [{'name': 'To Record Only Water For Ten Days'},
            {'name': 'Shadows Collide With People'},
            {'name': 'The Empyrean'}],
 'name': 'John Frusciante'}
